# Exploratory Analysis: Weather Impact on NYC Mobility

This notebook explores relationships between weather conditions and mobility patterns:
- Joining taxi and CitiBike data with weather data
- Analyzing correlations between weather and trip counts
- Comparing taxi vs. bike usage patterns
- Identifying weather-driven trends
- Discovering insights for further analysis

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from datetime import datetime

# Set visualization defaults
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Connect to DuckDB
db_path = Path('../data/nyc_mobility.duckdb')
conn = duckdb.connect(str(db_path), read_only=True)

print(f"Connected to DuckDB at: {db_path}")

## 1. Hourly Trip Counts by Transportation Mode

In [ ]:
# Aggregate hourly trip counts for each mode
hourly_trips = conn.execute("""
    WITH yellow_hourly AS (
        SELECT 
            DATE_TRUNC('hour', tpep_pickup_datetime) as hour,
            COUNT(*) as yellow_trips
        FROM raw_data.yellow_taxi
        GROUP BY hour
    ),
    fhv_hourly AS (
        SELECT 
            DATE_TRUNC('hour', pickup_datetime) as hour,
            COUNT(*) as fhv_trips
        FROM raw_data.fhv_taxi
        GROUP BY hour
    ),
    citibike_hourly AS (
        SELECT 
            DATE_TRUNC('hour', started_at) as hour,
            COUNT(*) as citibike_trips
        FROM raw_data.trips
        GROUP BY hour
    )
    SELECT 
        COALESCE(y.hour, f.hour, c.hour) as hour,
        COALESCE(y.yellow_trips, 0) as yellow_trips,
        COALESCE(f.fhv_trips, 0) as fhv_trips,
        COALESCE(c.citibike_trips, 0) as citibike_trips
    FROM yellow_hourly y
    FULL OUTER JOIN fhv_hourly f ON y.hour = f.hour
    FULL OUTER JOIN citibike_hourly c ON COALESCE(y.hour, f.hour) = c.hour
    ORDER BY hour
""").df()

print(f"Hourly trip data aggregated: {len(hourly_trips)} hours")
print(hourly_trips.head())

## 2. Join with Weather Data

In [ ]:
# Join trip counts with weather data
trips_weather = conn.execute("""
    WITH hourly_trips AS (
        SELECT 
            DATE_TRUNC('hour', tpep_pickup_datetime) as hour,
            COUNT(*) as yellow_trips
        FROM raw_data.yellow_taxi
        GROUP BY hour
    ),
    fhv_hourly AS (
        SELECT 
            DATE_TRUNC('hour', pickup_datetime) as hour,
            COUNT(*) as fhv_trips
        FROM raw_data.fhv_taxi
        GROUP BY hour
    ),
    citibike_hourly AS (
        SELECT 
            DATE_TRUNC('hour', started_at) as hour,
            COUNT(*) as citibike_trips
        FROM raw_data.trips
        GROUP BY hour
    )
    SELECT 
        DATE_TRUNC('hour', w.timestamp) as hour,
        COALESCE(yt.yellow_trips, 0) as yellow_trips,
        COALESCE(ft.fhv_trips, 0) as fhv_trips,
        COALESCE(ct.citibike_trips, 0) as citibike_trips,
        w.temp,
        w.feels_like,
        w.humidity,
        w.wind_speed,
        w.precipitation,
        w.rain,
        w.snowfall
    FROM raw_data.hourly_weather w
    LEFT JOIN hourly_trips yt ON DATE_TRUNC('hour', w.timestamp) = yt.hour
    LEFT JOIN fhv_hourly ft ON DATE_TRUNC('hour', w.timestamp) = ft.hour
    LEFT JOIN citibike_hourly ct ON DATE_TRUNC('hour', w.timestamp) = ct.hour
    ORDER BY hour
""").df()

print(f"Joined dataset: {len(trips_weather)} hourly records")
print(f"Columns: {trips_weather.columns.tolist()}")
print("\nSample data:")
display(trips_weather.head(10))

## 3. Correlation Analysis

In [ ]:
# Calculate correlations between weather and trip counts
correlation_cols = ['temp', 'feels_like', 'humidity', 'wind_speed', 
                    'yellow_trips', 'fhv_trips', 'citibike_trips']

corr_matrix = trips_weather[correlation_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix: Weather vs. Trip Counts')
plt.tight_layout()
plt.show()

print("\nKey Correlations with Temperature:")
print(f"  Yellow Taxi: {corr_matrix.loc['temp', 'yellow_trips']:.3f}")
print(f"  FHV: {corr_matrix.loc['temp', 'fhv_trips']:.3f}")
print(f"  CitiBike: {corr_matrix.loc['temp', 'citibike_trips']:.3f}")

## 4. Temperature vs. Trip Counts

In [ ]:
# Scatter plots: Temperature vs. trip counts
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Yellow Taxi
axes[0].scatter(trips_weather['temp'], trips_weather['yellow_trips'], 
               alpha=0.3, s=10)
axes[0].set_xlabel('Temperature (°C)')
axes[0].set_ylabel('Yellow Taxi Trips')
axes[0].set_title('Yellow Taxi vs. Temperature')
axes[0].grid(True, alpha=0.3)

# FHV
axes[1].scatter(trips_weather['temp'], trips_weather['fhv_trips'], 
               alpha=0.3, s=10, color='orange')
axes[1].set_xlabel('Temperature (°C)')
axes[1].set_ylabel('FHV Trips')
axes[1].set_title('FHV vs. Temperature')
axes[1].grid(True, alpha=0.3)

# CitiBike
axes[2].scatter(trips_weather['temp'], trips_weather['citibike_trips'], 
               alpha=0.3, s=10, color='green')
axes[2].set_xlabel('Temperature (°C)')
axes[2].set_ylabel('CitiBike Trips')
axes[2].set_title('CitiBike vs. Temperature')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Trips by Precipitation Level

In [ ]:
# Average trips by precipitation level
trips_weather['precipitation_category'] = pd.cut(
    trips_weather['precipitation'],
    bins=[-0.1, 0, 2.5, 7.5, 100],
    labels=['No Precipitation', 'Light (< 2.5mm)', 'Moderate (2.5-7.5mm)', 'Heavy (> 7.5mm)']
)

trips_by_weather = trips_weather.groupby('precipitation_category').agg({
    'yellow_trips': 'mean',
    'fhv_trips': 'mean',
    'citibike_trips': 'mean',
    'hour': 'count'
}).rename(columns={'hour': 'hours_observed'}).reset_index()

trips_by_weather = trips_by_weather.sort_values('hours_observed', ascending=False)

print("Average Trips by Precipitation Level:")
display(trips_by_weather)

# Plot average trips by precipitation level
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(trips_by_weather))
width = 0.25

ax.bar(x - width, trips_by_weather['yellow_trips'], width, 
       label='Yellow Taxi', color='steelblue')
ax.bar(x, trips_by_weather['fhv_trips'], width, 
       label='FHV', color='orange')
ax.bar(x + width, trips_by_weather['citibike_trips'], width, 
       label='CitiBike', color='green')

ax.set_xlabel('Precipitation Level')
ax.set_ylabel('Average Hourly Trips')
ax.set_title('Average Trips by Precipitation Level')
ax.set_xticks(x)
ax.set_xticklabels(trips_by_weather['precipitation_category'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Taxi vs. Bike Usage Patterns

In [ ]:
# Calculate daily totals
daily_trips = trips_weather.copy()
daily_trips['date'] = pd.to_datetime(daily_trips['hour']).dt.date

daily_totals = daily_trips.groupby('date').agg({
    'yellow_trips': 'sum',
    'fhv_trips': 'sum',
    'citibike_trips': 'sum',
    'temp': 'mean'
}).reset_index()

# Calculate ratio of bike to taxi trips
daily_totals['total_taxi'] = daily_totals['yellow_trips'] + daily_totals['fhv_trips']
daily_totals['bike_to_taxi_ratio'] = (daily_totals['citibike_trips'] / 
                                       daily_totals['total_taxi'])

# Plot time series
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Daily trip counts
axes[0].plot(daily_totals['date'], daily_totals['yellow_trips'], 
            label='Yellow Taxi', linewidth=2)
axes[0].plot(daily_totals['date'], daily_totals['fhv_trips'], 
            label='FHV', linewidth=2)
axes[0].plot(daily_totals['date'], daily_totals['citibike_trips'], 
            label='CitiBike', linewidth=2)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Daily Trips')
axes[0].set_title('Daily Trip Counts by Mode (Oct-Nov 2025)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bike to taxi ratio with temperature
ax2 = axes[1].twinx()
axes[1].plot(daily_totals['date'], daily_totals['bike_to_taxi_ratio'], 
            color='green', linewidth=2, label='Bike/Taxi Ratio')
ax2.plot(daily_totals['date'], daily_totals['temp'], 
         color='red', linewidth=1, alpha=0.5, label='Temperature')

axes[1].set_xlabel('Date')
axes[1].set_ylabel('Bike to Taxi Ratio', color='green')
ax2.set_ylabel('Temperature (°C)', color='red')
axes[1].set_title('CitiBike vs. Taxi Usage Ratio with Temperature')
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 7. Hour-of-Day Patterns by Weather

In [ ]:
# Extract hour of day
trips_weather['hour_of_day'] = pd.to_datetime(trips_weather['hour']).dt.hour

# Split into dry vs. rainy weather (precipitation > 0)
dry_weather = trips_weather[trips_weather['precipitation'] == 0]
rainy_weather = trips_weather[trips_weather['precipitation'] > 0]

# Average trips by hour for each weather type
dry_hourly = dry_weather.groupby('hour_of_day')[['yellow_trips', 'fhv_trips', 'citibike_trips']].mean()
rainy_hourly = rainy_weather.groupby('hour_of_day')[['yellow_trips', 'fhv_trips', 'citibike_trips']].mean()

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Yellow Taxi
axes[0].plot(dry_hourly.index, dry_hourly['yellow_trips'], 
            label='Dry', marker='o', linewidth=2)
if len(rainy_hourly) > 0:
    axes[0].plot(rainy_hourly.index, rainy_hourly['yellow_trips'], 
                label='Rain', marker='s', linewidth=2)
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Avg Trips per Hour')
axes[0].set_title('Yellow Taxi: Dry vs. Rain')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# FHV
axes[1].plot(dry_hourly.index, dry_hourly['fhv_trips'], 
            label='Dry', marker='o', linewidth=2, color='orange')
if len(rainy_hourly) > 0:
    axes[1].plot(rainy_hourly.index, rainy_hourly['fhv_trips'], 
                label='Rain', marker='s', linewidth=2, color='darkorange')
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Avg Trips per Hour')
axes[1].set_title('FHV: Dry vs. Rain')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# CitiBike
axes[2].plot(dry_hourly.index, dry_hourly['citibike_trips'], 
            label='Dry', marker='o', linewidth=2, color='green')
if len(rainy_hourly) > 0:
    axes[2].plot(rainy_hourly.index, rainy_hourly['citibike_trips'], 
                label='Rain', marker='s', linewidth=2, color='darkgreen')
axes[2].set_xlabel('Hour of Day')
axes[2].set_ylabel('Avg Trips per Hour')
axes[2].set_title('CitiBike: Dry vs. Rain')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Weekend vs. Weekday Patterns

In [ ]:
# Add weekday/weekend flag
trips_weather['dayofweek'] = pd.to_datetime(trips_weather['hour']).dt.dayofweek
trips_weather['is_weekend'] = trips_weather['dayofweek'].isin([5, 6])

# Compare weekend vs. weekday
weekend_avg = trips_weather[trips_weather['is_weekend']].groupby('hour_of_day')[[
    'yellow_trips', 'fhv_trips', 'citibike_trips'
]].mean()

weekday_avg = trips_weather[~trips_weather['is_weekend']].groupby('hour_of_day')[[
    'yellow_trips', 'fhv_trips', 'citibike_trips'
]].mean()

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

modes = ['yellow_trips', 'fhv_trips', 'citibike_trips']
titles = ['Yellow Taxi', 'FHV', 'CitiBike']
colors = ['steelblue', 'orange', 'green']

for i, (mode, title, color) in enumerate(zip(modes, titles, colors)):
    axes[i].plot(weekday_avg.index, weekday_avg[mode], 
                label='Weekday', marker='o', linewidth=2, color=color)
    axes[i].plot(weekend_avg.index, weekend_avg[mode], 
                label='Weekend', marker='s', linewidth=2, color=color, alpha=0.6)
    axes[i].set_xlabel('Hour of Day')
    axes[i].set_ylabel('Avg Trips per Hour')
    axes[i].set_title(f'{title}: Weekday vs. Weekend')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Key Findings and Insights

In [ ]:
# Calculate some key metrics
temp_corr_citibike = corr_matrix.loc['temp', 'citibike_trips']
temp_corr_yellow = corr_matrix.loc['temp', 'yellow_trips']
temp_corr_fhv = corr_matrix.loc['temp', 'fhv_trips']

# Rain impact
dry_citibike_avg = trips_by_weather[trips_by_weather['precipitation_category'] == 'No Precipitation']['citibike_trips'].values[0]
if len(trips_by_weather[trips_by_weather['precipitation_category'] != 'No Precipitation']) > 0:
    rainy_citibike_avg = trips_by_weather[trips_by_weather['precipitation_category'] != 'No Precipitation']['citibike_trips'].mean()
    rain_impact_citibike = ((rainy_citibike_avg - dry_citibike_avg) / dry_citibike_avg * 100)
else:
    rain_impact_citibike = 0

print("\n" + "="*80)
print("KEY FINDINGS: WEATHER IMPACT ON NYC MOBILITY")
print("="*80)

print("\n1. Temperature Correlations:")
print(f"   - CitiBike correlation with temperature: {temp_corr_citibike:.3f}")
print(f"   - Yellow Taxi correlation: {temp_corr_yellow:.3f}")
print(f"   - FHV correlation: {temp_corr_fhv:.3f}")
print(f"   → CitiBike usage is most sensitive to temperature changes")

if rain_impact_citibike != 0:
    print("\n2. Precipitation Impact:")
    print(f"   - CitiBike trips change by {rain_impact_citibike:.1f}% during precipitation")
    print(f"   → Precipitation significantly affects bike usage")

print("\n3. Usage Patterns:")
print(f"   - FHV has highest overall volume (~{trips_weather['fhv_trips'].mean():.0f} trips/hour)")
print(f"   - Yellow Taxi: ~{trips_weather['yellow_trips'].mean():.0f} trips/hour")
print(f"   - CitiBike: ~{trips_weather['citibike_trips'].mean():.0f} trips/hour")

print("\n4. Temporal Insights:")
print(f"   - CitiBike shows strong commute peaks (8am, 5-6pm)")
print(f"   - Taxis/FHV more evenly distributed throughout day")
print(f"   - Weekend patterns differ significantly from weekdays")

print("\n5. Seasonal Trends:")
print(f"   - Oct-Nov 2025 data shows fall weather patterns")
print(f"   - CitiBike usage varies with temperature")
print(f"   - Taxi/FHV usage more stable across temperature range")

print("\n" + "="*80)
print("RECOMMENDATIONS FOR FURTHER ANALYSIS:")
print("="*80)
print("  1. Build predictive model for CitiBike demand based on weather")
print("  2. Analyze specific precipitation levels and their impact")
print("  3. Study mode substitution (do people switch from bikes to taxis in bad weather?)")
print("  4. Examine geographic patterns (which neighborhoods are most weather-sensitive?)")
print("  5. Extend analysis to full year to capture seasonal variations")
print("="*80)

In [ ]:
# Close connection
conn.close()
print("\nConnection closed.")
print("\nExploratory analysis complete! ✓")